# ThreadPoolExecutorによる非同期変数書き換え

In [9]:
import cProfile, pstats, io
import numpy as np
from pstats import SortKey
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
import concurrent.futures
import time
from numba import jit

start_time=time.time()
max_number = 1000
x = 2
y = 15

a_l=np.zeros((max_number,x,y))
array_length = 3
variables = np.array([k for k in range(array_length)])
print(f'initial variables={variables}')

#@jit
def thread_share(i, a_l):
    global variables
    a_l[i,0,0] = i
    time.sleep(0.5)
#    print(f'variables in thread{i} = {variables}')
    a_l[i,0,2:array_length+2] = variables
    variables = variables+1
    a_l[i,0,array_length+2:array_length+5] = variables
    return i

with ThreadPoolExecutor(max_workers=max_number) as executor:
    futures = {executor.submit(thread_share, i, a_l):i for i in range(max_number)}
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        iterator_i = futures[future]
        a_l[iterator_i,0,1] = i
        future.result()
#        print(f'i,iterator i = {i}, {iterator_i}')
np.set_printoptions(formatter={'float': '{:10.0f}'.format})      
print(f'a_l = \n{a_l[0:20]}')        
print(f"finish sequence\n")

print(f'bench time = {time.time()-start_time}')
#test_profiler.disable()
#s = io.StringIO()    
#sortkey = SortKey.CUMULATIVE
#ps = pstats.Stats(test_profiler, stream=s).sort_stats(sortkey)
#ps.print_stats(20) 
#sortkey = SortKey.TIME  
#ps = pstats.Stats(test_profiler, stream=s).sort_stats(sortkey)
#ps.print_stats(20) 
#ps.print_callers(.5)
#print(s.getvalue())
#print(a_l)


       
#test_profiler.disable()
#with open("test.prof",mode="a+") as test_profile: 

    #sortkey = SortKey.CUMULATIVE
    #ps = pstats.Stats(test_profiler, stream=test_profile).sort_stats(sortkey)
    #ps.print_stats() 
    #sortkey = SortKey.TIME  
    #ps = pstats.Stats(test_profiler, stream=test_profile).sort_stats(sortkey)
    #ps.print_callers(.5, 'init')
#s = io.StringIO()    
#sortkey = SortKey.CUMULATIVE
#ps = pstats.Stats(test_profiler, stream=s).sort_stats(sortkey)
#ps.print_stats(20) 
#sortkey = SortKey.TIME  
#ps = pstats.Stats(test_profiler, stream=s).sort_stats(sortkey)
#ps.print_stats(20) 
#ps.print_callers(.5)
#print(s.getvalue())
# with np.load('filename.npz', allow_pickle=True) as loaded_records:
#    dimens = loaded_records['dimens']
#    matrices = loaded_records['matrices']


initial variables=[0 1 2]
a_l = 
[[[         0         12         13         14         15         14
           15         16          0          0          0          0
            0          0          0]
  [         0          0          0          0          0          0
            0          0          0          0          0          0
            0          0          0]]

 [[         1         25         25         26         27         26
           27         28          0          0          0          0
            0          0          0]
  [         0          0          0          0          0          0
            0          0          0          0          0          0
            0          0          0]]

 [[         2         19         20         21         22         21
           22         23          0          0          0          0
            0          0          0]
  [         0          0          0          0          0          0
            0      

# Threadとqueue

- キューの使い方安定しない

In [ ]:
import random
import queue
import threading
import time

# Threadクラスを継承し、所望のスレッド処理を実装する
class MyThread(threading.Thread):

  def __init__(self, x, y, queue):
    threading.Thread.__init__(self)
    self._x = x
    self._y = y
    self._queue = queue

  def run(self):
    self._queue.put((self._x, self._y, self._x + self._y))
    time.sleep(random.randint(1, 10))

queue = queue.Queue() # 入力と計算結果を投入するためのキュー
threads = [] # 生成したスレッドオブジェクトを格納するためのリスト

# スレッドオブジェクトを生成し、スレッドを開始する
for i in range(10):
  t = MyThread(random.randint(1,10), random.randint(1,10), queue)
  t.start()
  print('Thread started ', t)
  threads.append(t)

# スレッドの終了を待ち合わせる
for t in threads:
  t.join()

# キューから計算結果を取り出し表示する
while not queue.empty():
  result = queue.get()
  print(result)

- 同期キュー　

In [ ]:
import threading
import queue
import time
start_t = time.time()
q = queue.Queue()
max_worker = 5
a_l = []

def worker():
    global a_l
    while True:
        item = q.get()
        print(f'Working on {item}')
        time.sleep(0.1)
        a_l.append(item)
        print(f'Finished {item}')
        q.task_done()

# Turn-on the worker thread.
threading.Thread(target=worker, daemon=True).start()
# Send thirty task requests to the worker.
for item in range(max_worker):
    q.put(item)
# Block until all tasks are done.
q.join()
print(a_l)
print(time.time()-start_t)
print('All work completed')



- 非同期キュー　変数書き換え こちらが良い

In [ ]:
import threading
import queue
import time
import numpy as np
start_t = time.time()
q = queue.Queue()
threads = []
max_worker = 10
#a_l = []
a_l = np.empty([max_worker])

def worker(q, a_l):
    while True:
        item = q.get()
        if item is None:
            break
        print(f'Working on {item}')
        time.sleep(0.1)
#        a_l.append(item)
        a_l[item] = item
        print(f'Finished {item}')
        q.task_done()

# Turn-on the worker thread.
for i in range(max_worker):
    t = threading.Thread(target=worker, args=(q, a_l, ), daemon=True)
    t.start()
    threads.append(t)
for item in range(max_worker):
    q.put(item, a_l)

for item in range(max_worker):
    q.put(None)
# Block until all tasks are done.
for t in threads:
    t.join()
print(a_l)
print(time.time()-start_t)
print('All work completed')



In [ ]:
import threading
import queue
import time
import numpy as np
start_t = time.time()
q = queue.Queue()
threads = []
max_worker = 10
#a_l = []

var = np.array([i for i in range(3)])
a_l = np.empty([max_worker,2+len(var)])

def worker(q, a_l, var):
    while True:
        item = q.get()
        if item is None:
            break
        print(f'Working on {item}')
        time.sleep(0.5)
        var = var*2
        print(var)    
#        a_l.append(item)
        a_l[item,0] = item
        a_l[item,1:len(var)+1] = var
        print(f'Finished {item}')
        q.task_done()
    return item
# Turn-on the worker thread.
for i in range(max_worker):
    t = threading.Thread(target=worker, args=(q, a_l, var ), daemon=True)
    t.start()
    threads.append(t)
for item in range(max_worker):
    q.put(item, a_l, var)

for item in range(max_worker):
    q.put(None)
# Block until all tasks are done.
for t in threads:
    t.join()
print(a_l)
print(time.time()-start_t)
print('All work completed')



In [ ]:
import threading
import queue
import time
import numpy as np

start_t = time.time()
q = queue.Queue()
threads = []
max_worker = 10
#a_l = []

var = np.array([i for i in range(3)])
a_l = np.empty([max_worker,2+len(var)])

def worker(q, a_l):
    global var
    while True:
        item = q.get()
        if item is None:
            break
        print(f'Working on {item}')
        time.sleep(0.5)
        var = var*2
        print(var)    
#        a_l.append(item)
        a_l[item,0] = item
        a_l[item,1:len(var)+1] = var
        print(f'Finished {item}')
        q.task_done()
    return item
# Turn-on the worker thread.
for i in range(max_worker):
    t = threading.Thread(target=worker, args=(q, a_l), daemon=True)
    t.start()
    threads.append(t)
for item in range(max_worker):
    q.put(item, a_l)

for item in range(max_worker):
    q.put(None)
# Block until all tasks are done.
for t in threads:
    t.join()
np.set_printoptions(precision=3)
print(a_l)
print(time.time()-start_t)
print('All work completed')



In [ ]:
import threading
import queue
import time

q = queue.Queue()
thread = []
max_worker = 5
def worker():
    while True:
        item = q.get()
        if item is None:
            break
        print(f'Working on {item}')
        time.sleep(0.1)
        print(f'Finished {item}')
        q.task_done()

# Turn-on the worker thread.
for i in range(max_worker):
    t = threading.Thread(target=worker, daemon=True)
    t.start()
    thread.append(t)
for item in range(max_worker):
    q.put(item)
# Send thirty task requests to the worker.
#for item in range(max_worker):
#    q.put(None)
# Block until all tasks are done.
q.join()
print('All work completed')



- 同期キューで変数書き換え

In [ ]:
import threading
import queue
import time
import numpy as np
start_t = time.time()
q = queue.Queue()
max_number = 10
x = 2
y = 15
#test_profiler=cProfile.Profile()
#test_profiler.enable()
a_l=np.zeros((max_number,x,y))
array_length = 3
variables = np.array([k for k in range(array_length)])
print(variables)
#@jit
def thread_share(q, a_l):
    global variables
    while True:
        item = q.get()
        #if i is None:
        #    break
        time.sleep(0.5)
        a_l[item,0,0] = item
        print(variables)
        a_l[item,0,2:array_length+2] = variables
        variables = variables*2
        a_l[item,0,array_length+2:array_length+5] = variables
        q.task_done()

# Turn-on the worker thread.
#threading.Thread(target=thread_share, args=(q, a_l,), daemon=True).start()
threading.Thread(target=thread_share, args=(q, a_l), daemon=True).start()
# Send thirty task requests to the worker.
for i in range(max_number):
    q.put(i, a_l)
# Block until all tasks are done.
q.join()
np.set_printoptions( precision=3)
print(a_l)
print(time.time()-start_t)
print('All work completed')



- 非同期　変数書き換え

In [ ]:
import threading
import queue
import time
import numpy as np
from numba import jit
start_t = time.time()
q = queue.Queue()
max_number = 10
x = 2
y = 15
a_l=np.zeros((max_number,x,y))
array_length = 3
variables = np.array([k for k in range(array_length)])
print(variables)
threads = np.empty([max_number]).astype(object)

#@jit (forceobj=True)
def thread_share(q, a_l):
    global variables
    while True:
        item = q.get()    
        if item is None:
            break
#        print(a_l.shape)
        time.sleep(0.5)
#        print(f'item = {item}')
        a_l[item,0,0] = item
#        print(variables)
        a_l[item,0,2:array_length+2] = variables
        variables = variables+1
        a_l[item,0,array_length+2:array_length+5] = variables
        q.task_done()
    return item

for i in range(max_number):
    t = threading.Thread(target=thread_share, args=(q, a_l), daemon=True)
    t.start()
    threads[i]=t
for item in range(max_number):
    q.put(item, a_l)
for item in range(max_number):
    q.put(None)
for t in threads:
    t.join()
np.set_printoptions(formatter={'float': '{:10.0f}'.format})
print(a_l[0:20])
print(time.time()-start_t)
print('All work completed')


# asyncio

- 同期処理 await asyncio.sleepをtime.sleepにすると同期処理になる

In [ ]:
import asyncio
import time
import numpy as np
start_t = time.time()
max_number = 10
x = 2
y = 15
a_l=np.zeros((max_number,x,y))
print(a_l.shape)
array_length = 3
variables = np.array([k for k in range(array_length)])
print(variables)

#@jit
async def thread_share(q, a_l):
    global variables
    time.sleep(0.5)
    print(f'q={q}')
    a_l[q,0,0] = q
    print(variables)
    a_l[q,0,2:array_length+2] = variables
    variables = variables*2
    a_l[q,0,array_length+2:array_length+5] = variables
    return q

async def main():
    tasks = [asyncio.create_task(thread_share(q, a_l)) for q in range(max_number)]
    r = [await task for task in tasks]
    return r  

await main()
np.set_printoptions(formatter={'float': '{:5.0f}'.format})
print(a_l)
print(time.time()-start_t)
print('All work completed')


- 非同期 引数渡し
    - jit（numba)はawaitを展開できない

In [ ]:
import asyncio
import time
import numpy as np
from numba import jit # 使用できない
start_t = time.time()
max_number = 1000
max_workers = 3
x = 2
y = 15
a_l=np.zeros((max_number,x,y))

array_length = 3
variables = np.array([k for k in range(array_length)])
print(variables)

#@jit #awaitが展開できずにエラーになる
async def thread_share( q, a_l ):
    global variables
    await asyncio.sleep(0.5)
    #time.sleep(0.5) こっちだと非同期になる
#    print(f'q={q}')
    a_l[q,0,0] = q
#    print(variables)
    a_l[q,0,2:array_length+2] = variables
    variables = variables+1
    a_l[q,0,array_length+2:array_length+5] = variables


async def main():
    tasks = (thread_share(q , a_l) for q in range(max_number))
    await asyncio.gather(*tasks)

await main()
np.set_printoptions(formatter={'float': '{:10.0f}'.format})
print(a_l[0:20])
print(time.time()-start_t)
print('All work completed')

In [ ]:
import asyncio
import time
import profile
import random
async def calc(x):
    r = 0
    for i in range(10**7):
        if random.random() < 0.99:
            r += i % x
    return r

async def main():
    tasks = [asyncio.create_task(calc(i)) for i in range(1, 16)]
    # この書き方がmainの中のthreadも進行できて良い
    # ここになにか処理を書くのがasyncioのプラクティス
    r = [await task for task in tasks]
    print(r)

start = time.time()
await main()
elapsed = time.time() - start
print(elapsed)

In [ ]:
import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

#asyncio.run(main())
await main()

In [ ]:
import asyncio
import time


def task(tid, n):
    print(f'Task {tid} started')
    time.sleep(n)
    print(f'Task {tid} finished')
    return n

async def main():
    loop = asyncio.get_running_loop()

    result = [await loop.run_in_executor(None, task, n, 0.5) for n in range(5)]
    print('default thread pool', result)

await main()